In [1]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from transformers import BertForSequenceClassification, AdamW
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from transformers import BertForSequenceClassification, AdamW
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import urllib.request
from tqdm import tqdm
from transformers import BertTokenizer, TFBertForSequenceClassification
import logging
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import numpy as np
import pickle
import re
import urllib.request
from tqdm import tqdm
from transformers import BertTokenizer, TFBertForSequenceClassification
import pandas_datareader.data as web # 주식 데이터를 얻어오기 위해 사용
import datetime # 시간 처리
import matplotlib.pyplot as plt
import yfinance as yf
import FinanceDataReader as fdr
from prophet import Prophet

C:\Users\chan\anaconda3\envs\ng\lib\site-packages\yfinance\base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [2]:
def bertcode_predict(ticker, start_date, end_date_1):
    ticker = ticker
    #start_date 21-04-06 / end_date_1 24-05-09
    start_date = start_date
    end_date = end_date_1

    start_year, start_month, start_day = start_date.split('-')
    end_year, end_month, end_day = end_date.split('-')
    
    start_year = int(start_year)
    start_month = int(start_month)
    start_day = int(start_day)

    end_year = int(end_year)
    end_month = int(end_month)
    end_day = int(end_day)
    


    
    max_seq_len = 128
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    path = f"C:/Users/chan/Documents/GitHub/ML/test_{ticker}/"
    # path = "C:/Users/chan/Documents/GitHub/ML/test10_AVGO/"
    news_df = pd.DataFrame(columns=["title", "content"])

    
    for txts in os.listdir(path):
        full_path = os.path.join(path, txts)  # 파일 전체 경로 생성
        if full_path.endswith('header.txt'):
            continue
        if os.path.isfile(full_path):  # 파일인지 확인
            with open(full_path, "r", encoding="utf-8") as txt_file:
                title = txt_file.readline().strip()
                content = txt_file.read().replace('\n', ' ')
                # DataFrame 생성 후 concat 함수를 사용하여 추가
                new_row = pd.DataFrame({"title": [title], "content": [content]})
                news_df = pd.concat([news_df, new_row], ignore_index=True)
    inputs = [content for content in news_df['content']]

    # 입력 데이터를 BERT 모델의 입력 형식에 맞게 변환
    max_length = 128
    input_ids = []
    attention_masks = []
    
    for content in inputs:
        encoded_dict = tokenizer.encode_plus(
                            content,                    # content
                            add_special_tokens = True,  # Add '[CLS]' and '[SEP]'
                            max_length = max_length,           # Pad & truncate all sentences
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks
                            return_tensors = 'pt',     # Return pytorch tensors
                       )
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    predict_model = torch.load("bert_model_loss0.34.pth", map_location=torch.device('cpu'))
    predict_model.to(torch.device('cpu'))

    predicted_labels = []

    for inputs in tqdm(zip(input_ids, attention_masks)):
        input_ids = inputs[0].to(torch.device('cpu'))
        attention_mask = inputs[1].to(torch.device('cpu'))
        output = predict_model(input_ids, attention_mask=attention_mask)
    
        ps = F.softmax(output.logits, dim=1)
        top_p, top_class = ps.topk(1, dim=1)
        predicted_labels.append(top_class.item())
    
    predict_df = pd.DataFrame({'predicted_label': predicted_labels})
    news_df["predict"] = predict_df
    news_df.to_csv('predicted_news.csv', index=False)

    news_df['date'] = pd.to_datetime(news_df['title']).dt.date  # 날짜 추출
    news_df['time'] = pd.to_datetime(news_df['title']).dt.time  # 시간 추출
    
    # 날짜별로 그룹화하고, 'predict' 열에 대한 평균을 계산
    grouped_df = news_df.groupby('date').agg({
        'title': lambda x: x.tolist(),
        'content': lambda x: x.tolist(),
        'predict': lambda x: round(x.mean(), 3)  
    })
    
    grouped_df.reset_index(inplace=True)  # 인덱스 리셋
    
    # 결과 출력
    grouped_df[['date', 'title', 'content', 'predict']]

    result_df = grouped_df[['date', 'predict']]

    news_df['date'] = pd.to_datetime(news_df['title']).dt.date  # 날짜 추출
    news_df['time'] = pd.to_datetime(news_df['title']).dt.time  # 시간 추출
    
    # 'date' 열을 datetime 형식으로 다시 변환
    news_df['date'] = pd.to_datetime(news_df['date'])
    
    # 날짜별로 그룹화하고, 'predict' 열에 대한 평균을 계산
    grouped_df = news_df.groupby('date').agg({
        'title': lambda x: x.tolist(),
        'content': lambda x: x.tolist(),
        'predict': lambda x: round(x.mean(), 3)  
    })
    
    grouped_df.reset_index(inplace=True)  # 인덱스 리셋
    
    # 모든 날짜를 포함하는 날짜 범위 생성
    all_dates = pd.date_range(start=grouped_df['date'].min(), end=grouped_df['date'].max(), freq='D')
    # 새로운 DataFrame 생성 후 기존 데이터와 병합
    complete_df = pd.DataFrame(all_dates, columns=['date'])
    complete_df['date'] = pd.to_datetime(complete_df['date'])  # 날짜를 datetime으로 변환
    complete_df = complete_df.merge(grouped_df, on='date', how='left')
    # 누락된 'predict' 값을 이전 값으로 채우기
    complete_df['predict'] = complete_df['predict'].fillna(method='ffill')
    
    # 'date'와 'predict' 열만 선택하여 결과 출력
    result_df = complete_df[['date', 'predict']]

    start_date = datetime.datetime( start_year, start_month, start_day )
    end_date = datetime.datetime( end_year, end_month, end_day+1)
    
    # 주어진 범위 내에 있는 날짜 생성
    missing_dates = pd.date_range(start=start_date, end=end_date).difference(result_df['date'])
    
    # 새로운 날짜와 예측값 추가
    for date in missing_dates:
        result_df = result_df.append({'date': date, 'predict': 1.0}, ignore_index=True)
    
    # 날짜 기준으로 데이터프레임 정렬
    result_df = result_df.sort_values('date').reset_index(drop=True)

    def makeStockChart(code, sDay, eDay):
        data = yf.download(code, start=sDay, end=eDay)

    code_name = ticker
    makeStockChart(code_name, start_date, end_date) #기간받아오기
    
    datas = yf.download(code_name, start=start_date, end=end_date)

    dic = {
        'ds' : datas.index,
        'y' : datas.Close,
        # 'volume': datas.Volume
        
    }
    
    finance_df = pd.DataFrame( dic )
    
    #인덱스 초기화(원본 까지 적용)
    finance_df.reset_index( inplace=True )
    
    del finance_df['Date']

    date_range = pd.date_range(start=start_date, end=end_date)
    df_date_range = pd.DataFrame(date_range, columns=['date'])
    df_filtered = df_date_range[~df_date_range['date'].isin(finance_df['ds'])]

    common_dates = result_df[result_df['date'].isin(df_filtered['date'])]['date']
    result_filtered = result_df[~result_df['date'].isin(common_dates)]

    result_filtered.reset_index(inplace=True)

    result_filtered_future = result_filtered.copy()
    result_filtered_future = result_filtered_future.drop(columns=['index'])

    # 'predict' 열에만 평균 값 추가
    mean_last_seven = result_filtered_future['predict'].iloc[-7:].mean()
    # mean_last_seven_1 = result_filtered_future['volume'].iloc[-7:].mean()
    # 다음 날짜 계산
    last_date = result_filtered_future['date'].iloc[-1]
    next_date = pd.to_datetime(last_date) + pd.DateOffset(days=1)
    
    # 데이터프레임에 새로운 행 추가
    new_row = {'date': next_date, 'predict': round(mean_last_seven, 3)}  # 소수점 세 자리까지 반올림
    result_filtered_future = result_filtered_future.append(new_row, ignore_index=True)
    
    # 'predict' 열의 소수점 자리 수 설정
    result_filtered_future['predict'] = result_filtered_future['predict'].round(3)
    
    finance_df = pd.concat([finance_df, result_filtered['predict']], axis=1)

    return finance_df, result_filtered_future

In [3]:
def prophet_model_predict(ticker, start_date, end_date_1):
    
    # 저장된 모델 파라미터를 불러오는 함수
    ticker = ticker
    start_date = start_date
    end_date_1 = end_date_1 
    finance_df,result_filtered_future = bertcode_predict(ticker,start_date,end_date_1)
    def load_prophet_model_parameters(file_path):
        with open(file_path, 'rb') as f:
            model_params = pickle.load(f)
        return model_params
        
    file_path = f"prophet_model_parameters_{ticker}.pkl"
    loaded_model_parameters = load_prophet_model_parameters(file_path)
    
    # Prophet 모델 생성 및 파라미터 설정
    loaded_model = Prophet(
        changepoint_prior_scale=loaded_model_parameters['changepoint_prior_scale'],
        seasonality_prior_scale=loaded_model_parameters['seasonality_prior_scale'],
        holidays_prior_scale=loaded_model_parameters['holidays_prior_scale'],
        seasonality_mode=loaded_model_parameters['seasonality_mode'],
        changepoint_range=loaded_model_parameters['changepoint_range'],
        yearly_seasonality=loaded_model_parameters['yearly_seasonality'],
        weekly_seasonality=loaded_model_parameters['weekly_seasonality'],
        daily_seasonality=loaded_model_parameters['daily_seasonality'],
        growth=loaded_model_parameters['growth'],
        n_changepoints=loaded_model_parameters['n_changepoints']
    )
    loaded_model.add_country_holidays(country_name='US')
    
    loaded_model.add_regressor('predict')
    
    # 데이터 학습
    loaded_model.fit(finance_df)
    future = loaded_model.make_future_dataframe( periods = 1)
    future['predict'] = result_filtered_future['predict'].values
    # 주가 예측
    forecast = loaded_model.predict(future)

    return finance_df, forecast 

In [4]:
def simulate_trading(ticker, start_date, end_date_1, end_date_2,initial_capital):

    start_date = start_date
    end_date_1 = end_date_1
    end_date_2 = end_date_2
    finance_df,forecast = prophet_model_predict(ticker, start_date, end_date_1)
    # 입력된 날짜 범위로 데이터 필터링
    filtered_df = finance_df[(finance_df['ds'] >= start_date) & (finance_df['ds'] <= end_date_1)]
    filtered_df_forecast = forecast[(forecast['ds'] >= start_date) & (forecast['ds'] <= end_date_2)]

    # 현재 자본 및 보유 주식 수 초기화
    current_capital = initial_capital
    shares_owned = 0

    # 현재 포지션 초기화
    position = 'None'

    # 실제 및 예측 주식 가격 설정
    actual_prices = filtered_df.set_index('ds')['y']
    predicted_prices = filtered_df_forecast.set_index('ds')['yhat']

    # 거래 기록 초기화
    trading_history = []

    # 시뮬레이션 시작
    for date, today_price, predicted_next_day_price in zip(actual_prices.index, actual_prices.values, predicted_prices.shift(-1).values):
        predicted_change = predicted_next_day_price - today_price

        # 매수 조건
        if predicted_change > 0 and position != 'Hold':
            shares_to_buy = current_capital / today_price
            shares_owned += shares_to_buy
            current_capital -= shares_to_buy * today_price
            position = 'Hold'
            trading_history.append('Buy')

        # 매도 조건
        elif predicted_change < 0 and position == 'Hold':
            current_capital += shares_owned * today_price
            shares_owned = 0
            position = 'None'
            trading_history.append('Cell')

        # 보유 조건
        elif position == 'Hold' and abs(predicted_change) <= 0.01 * today_price:
            trading_history.append('Hold')

        # 조치 없음 또는 보유
        else:
            if position == 'Hold':
                trading_history.append('Hold')
            else:
                trading_history.append('Stay')#조치없음

    # 최종 자본 및 수익률 계산

    # trading_history의 마지막 인덱스 반환
    last_record = trading_history[-1]
    return last_record

In [5]:
ticker = 'AVGO'
start_date = '2021-04-06'
end_date = '2024-05-14'
end_date_future = '2024-05-15'
initial_capital = 10000000000.0
state = simulate_trading(ticker, start_date, end_date, end_date_future, initial_capital)
print(state)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\chan\anaconda3\envs\ng\lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
998it [01:06, 15.09it/s]
C:\Users\chan\AppData\Local\Temp\ipykernel_4060\1829546400.py:13

Cell


In [5]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from transformers import BertForSequenceClassification, AdamW
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import urllib.request
from tqdm import tqdm
from transformers import BertTokenizer, TFBertForSequenceClassification

import pandas_datareader.data as web # 주식 데이터를 얻어오기 위해 사용
import datetime # 시간 처리
import yfinance as yf
import FinanceDataReader as fdr
from prophet import Prophet
# %matplotlib inline

ticker = 'AVGO'
start_date = '2021-04-05'
end_date = '2024-05-16'
counter = 1
# def bertcode_predict(ticker, start_date, end_date_1,counter):
ticker = ticker
counter = counter
#start_date 21-04-06 / end_date_1 24-05-09
start_date = start_date
end_date = end_date

start_year, start_month, start_day = start_date.split('-')
end_year, end_month, end_day = end_date.split('-')

start_year = int(start_year)
start_month = int(start_month)
start_day = int(start_day)

end_year = int(end_year)
end_month = int(end_month)
end_day = int(end_day)




max_seq_len = 128
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

path = f"C:/Users/chan/Documents/GitHub/ML/test_{ticker}/"
# path = "C:/Users/chan/Documents/GitHub/ML/test10_AVGO/"
news_df = pd.DataFrame(columns=["title", "content"])


for txts in os.listdir(path):
    full_path = os.path.join(path, txts)  # 파일 전체 경로 생성
    if full_path.endswith('header.txt'):
        continue
    if os.path.isfile(full_path):  # 파일인지 확인
        with open(full_path, "r", encoding="utf-8") as txt_file:
            title = txt_file.readline().strip()
            content = txt_file.read().replace('\n', ' ')
            # DataFrame 생성 후 concat 함수를 사용하여 추가
            new_row = pd.DataFrame({"title": [title], "content": [content]})
            news_df = pd.concat([news_df, new_row], ignore_index=True)
inputs = [content for content in news_df['content']]

# 입력 데이터를 BERT 모델의 입력 형식에 맞게 변환
max_length = 128
input_ids = []
attention_masks = []

for content in inputs:
    encoded_dict = tokenizer.encode_plus(
                        content,                    # content
                        add_special_tokens = True,  # Add '[CLS]' and '[SEP]'
                        max_length = max_length,           # Pad & truncate all sentences
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks
                        return_tensors = 'pt',     # Return pytorch tensors
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

predict_model = torch.load("bert_model_loss0.34.pth", map_location=torch.device('cpu'))
predict_model.to(torch.device('cpu'))

predicted_labels = []

for inputs in tqdm(zip(input_ids, attention_masks)):
    input_ids = inputs[0].to(torch.device('cpu'))
    attention_mask = inputs[1].to(torch.device('cpu'))
    output = predict_model(input_ids, attention_mask=attention_mask)

    ps = F.softmax(output.logits, dim=1)
    top_p, top_class = ps.topk(1, dim=1)
    predicted_labels.append(top_class.item())

predict_df = pd.DataFrame({'predicted_label': predicted_labels})
news_df["predict"] = predict_df
news_df.to_csv('predicted_news.csv', index=False)

news_df['date'] = pd.to_datetime(news_df['title']).dt.date  # 날짜 추출
news_df['time'] = pd.to_datetime(news_df['title']).dt.time  # 시간 추출

# 날짜별로 그룹화하고, 'predict' 열에 대한 평균을 계산
grouped_df = news_df.groupby('date').agg({
    'title': lambda x: x.tolist(),
    'content': lambda x: x.tolist(),
    'predict': lambda x: round(x.mean(), 3)  
})

grouped_df.reset_index(inplace=True)  # 인덱스 리셋

# 결과 출력
grouped_df[['date', 'title', 'content', 'predict']]

result_df = grouped_df[['date', 'predict']]

news_df['date'] = pd.to_datetime(news_df['title']).dt.date  # 날짜 추출
news_df['time'] = pd.to_datetime(news_df['title']).dt.time  # 시간 추출

# 'date' 열을 datetime 형식으로 다시 변환
news_df['date'] = pd.to_datetime(news_df['date'])

# 날짜별로 그룹화하고, 'predict' 열에 대한 평균을 계산
grouped_df = news_df.groupby('date').agg({
    'title': lambda x: x.tolist(),
    'content': lambda x: x.tolist(),
    'predict': lambda x: round(x.mean(), 3)  
})

grouped_df.reset_index(inplace=True)  # 인덱스 리셋

# 모든 날짜를 포함하는 날짜 범위 생성
all_dates = pd.date_range(start=grouped_df['date'].min(), end=grouped_df['date'].max(), freq='D')
# 새로운 DataFrame 생성 후 기존 데이터와 병합
complete_df = pd.DataFrame(all_dates, columns=['date'])
complete_df['date'] = pd.to_datetime(complete_df['date'])  # 날짜를 datetime으로 변환
complete_df = complete_df.merge(grouped_df, on='date', how='left')
# 누락된 'predict' 값을 이전 값으로 채우기
complete_df['predict'] = complete_df['predict'].fillna(method='ffill')

# 'date'와 'predict' 열만 선택하여 결과 출력
result_df_1 = complete_df[['date', 'predict']]

##############################################
csv_filename = f"news_data_{ticker}.csv"
loaded_df = pd.read_csv(csv_filename)

result_df_1['date'] = result_df_1['date'].dt.date
result_df = pd.concat([loaded_df, result_df_1], ignore_index=True)
print(result_df)

###############################################

start_date = datetime.datetime( start_year, start_month, start_day )
end_date = datetime.datetime( end_year, end_month, end_day)

# 주어진 범위 내에 있는 날짜 생성
# missing_dates = pd.date_range(start=start_date, end=end_date).difference(result_df['date'])

# # 새로운 날짜와 예측값 추가
# for date in missing_dates:
#     result_df = result_df.append({'date': date, 'predict': 1.0}, ignore_index=True)

# 날짜 기준으로 데이터프레임 정렬
result_df['date'] = pd.to_datetime(result_df['date'])
result_df = result_df.sort_values('date').reset_index(drop=True)
print(result_df)

if counter == 6:
    result_df.to_csv(csv_filename, index=False)
    
def makeStockChart(code, sDay, eDay):
    data = yf.download(code, start=sDay, end=eDay)

code_name = ticker
makeStockChart(code_name, start_date, end_date) #기간받아오기

datas = yf.download(code_name, start=start_date, end=end_date)

dic = {
    'ds' : datas.index,
    'y' : datas.Close,
    # 'volume': datas.Volume
    
}

finance_df = pd.DataFrame( dic )

#인덱스 초기화(원본 까지 적용)
finance_df.reset_index( inplace=True )

del finance_df['Date']

#########################
stock = yf.Ticker(ticker)
hist = stock.history(period = '1d',interval="1h")
last_row = finance_df.iloc[-1]  # 마지막 행 가져오기
next_date = last_row['ds'] + pd.DateOffset(days=1)
second_price = hist['Close'][counter]# 1 -> 10시반 / 3 -> 12시반 / 5 -> 2시반 / 7 -> 4시반
start_next_row = pd.DataFrame({'ds': [next_date], 'y': second_price })
finance_df = pd.concat([finance_df, start_next_row], ignore_index=True)

#########################

date_range = pd.date_range(start=start_date, end=end_date)
df_date_range = pd.DataFrame(date_range, columns=['date'])
df_filtered = df_date_range[~df_date_range['date'].isin(finance_df['ds'])]

common_dates = result_df[result_df['date'].isin(df_filtered['date'])]['date']
result_filtered = result_df[~result_df['date'].isin(common_dates)]

result_filtered.reset_index(inplace=True)

result_filtered_future = result_filtered.copy()
result_filtered_future = result_filtered_future.drop(columns=['index'])

# 'predict' 열에만 평균 값 추가
mean_last_seven = result_filtered_future['predict'].iloc[-7:].mean()
# mean_last_seven_1 = result_filtered_future['volume'].iloc[-7:].mean()
# 다음 날짜 계산
last_date = result_filtered_future['date'].iloc[-1]
next_date = pd.to_datetime(last_date) + pd.DateOffset(days=1)

# 데이터프레임에 새로운 행 추가
new_row = {'date': next_date, 'predict': round(mean_last_seven, 3)}  # 소수점 세 자리까지 반올림
result_filtered_future = result_filtered_future.append(new_row, ignore_index=True)

# 'predict' 열의 소수점 자리 수 설정
result_filtered_future['predict'] = result_filtered_future['predict'].round(3)

finance_df = pd.concat([finance_df, result_filtered['predict']], axis=1)
print(finance_df,result_filtered_future)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\chan\anaconda3\envs\ng\lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
6it [00:01,  4.25it/s]
C:\Users\chan\AppData\Local\Temp\ipykernel_17836\850901545.py:155:

            date  predict
0     2021-04-05    1.000
1     2021-04-06    1.000
2     2021-04-07    1.000
3     2021-04-08    1.000
4     2021-04-09    1.000
...          ...      ...
1133  2024-05-12    0.500
1134  2024-05-13    0.917
1135  2024-05-14    1.126
1136  2024-05-15    1.000
1137  2024-05-16    1.000

[1138 rows x 2 columns]
           date  predict
0    2021-04-05    1.000
1    2021-04-06    1.000
2    2021-04-07    1.000
3    2021-04-08    1.000
4    2021-04-09    1.000
...         ...      ...
1133 2024-05-12    0.500
1134 2024-05-13    0.917
1135 2024-05-14    1.126
1136 2024-05-15    1.000
1137 2024-05-16    1.000

[1138 rows x 2 columns]
            ds            y  predict
0   2021-04-05   488.480011    1.000
1   2021-04-06   483.869995    1.000
2   2021-04-07   482.459991    1.000
3   2021-04-08   485.480011    1.000
4   2021-04-09   485.089996    1.000
..         ...          ...      ...
781 2024-05-10  1332.800049    1.100
782 2024-05-13  1337.510010    0.917
783 2

C:\Users\chan\AppData\Local\Temp\ipykernel_17836\850901545.py:233: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_filtered_future = result_filtered_future.append(new_row, ignore_index=True)
